In [5]:
import numpy as np
import cv2 as cv
import os
from PIL import Image
import pickle
import face_recognition

# Region of Interest

In [ ]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt2.xml')

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv.flip(frame,1)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray,1.5,5)
    
    for(x,y,w,h) in faces: 
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = frame[y:y+h,x:x+w]
        img_item = "my-image.png"
        cv.imwrite(img_item,roi_gray)
        
        color = (0,255,0)
        stroke = 2
        cv.rectangle(frame,(x,y),(x+w,y+h),color,stroke)
        
    cv.imshow('frame',frame)
    if cv.waitKey(10) == 27:
        break
        
cap.release()
cv.destroyAllWindows()

# Try to recognize face using LBPH face recognizer but it's not working.

In [ ]:
BASE_DIR = os.path.dirname(os.path.abspath('FaceRecognition'))

image_dir = os.path.join(BASE_DIR,"images")


face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt2.xml')
recognizer = cv.face.LBPHFaceRecognizer_create()                         

current_id = 0
label_ids = {}
y_labels = []
x_trains = []

for root,dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root,file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
            #print(label,path)
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            #print(label_ids)
            
            
            pil_Image = Image.open(path).convert("L")
            image_array = np.array(pil_Image,"uint8")
            
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array,1.5,5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h,x:x+w]
                x_trains.append(roi)
                y_labels.append(id_)
            
with open("labels.pickle",'wb') as f:
    pickle.dump(label_ids,f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainner.yml")

In [ ]:
BASE_DIR

In [ ]:
os.path.dirname(path)

In [ ]:
path

In [3]:
cv.__version__

'4.8.0'

# Recognition using face_recognition library.

In [6]:
BASE_DIR = os.path.dirname(os.path.abspath('FaceRecognition'))
image_dir = os.path.join(BASE_DIR, "images")

known_encodings = []
y_labels = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
            
            pil_image = Image.open(path).convert("RGB")
            image_array = np.array(pil_image)
            
            # Use face_recognition to detect faces and compute encodings
            face_locations = face_recognition.face_locations(image_array)
            face_encodings = face_recognition.face_encodings(image_array, face_locations)
            
            for face_encoding in face_encodings:
                known_encodings.append(face_encoding)
                y_labels.append(label)

# Save the known encodings and labels to files
with open("encodings.pickle", 'wb') as f:
    pickle.dump(known_encodings, f)

with open("labels.pickle", 'wb') as f:
    pickle.dump(y_labels, f)


In [7]:
y_labels

['narendra-modi',
 'narendra-modi',
 'narendra-modi',
 'narendra-modi',
 'narendra-modi',
 'rohit-sharma',
 'rohit-sharma',
 'rohit-sharma',
 'rohit-sharma',
 'rohit-sharma',
 'swami-vivekanand',
 'swami-vivekanand',
 'swami-vivekanand',
 'swami-vivekanand',
 'swami-vivekanand',
 'swami-vivekanand',
 'virat-kohli',
 'virat-kohli',
 'virat-kohli',
 'virat-kohli',
 'virat-kohli']

In [ ]:
# Load known encodings and labels from files
with open("encodings.pickle", 'rb') as f:
    known_encodings = pickle.load(f)

with open("labels.pickle", 'rb') as f:
    y_labels = pickle.load(f)


video_capture = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_capture.read()
    
    frame = cv2.flip(frame,1)

    # Convert the image to RGB format
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Iterate over detected faces
    for face_encoding in face_encodings:
        # Compare the face encoding with known encodings
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Unknown"

        # Check if there is a match
        if True in matches:
            matched_indices = [i for i, match in enumerate(matches) if match]
            labels = [y_labels[i] for i in matched_indices]
            name = max(set(labels), key=labels.count)

        # Draw rectangle and label on the face
        top, right, bottom, left = face_recognition.face_locations(rgb_frame)[0]
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    
    cv2.imshow('Face Recognition', frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()
